In [15]:
# import required packages

import requests
import pandas as pd
import config

In [16]:
# pull overpass data from Spectator.Earth API

api_key = config.api_key 
bbox = '-6.15450317,53.30408682,-6.07896861,53.34080955' # Dublin Bay
satellites = 'Sentinel-1A, Sentinel-1B, Sentinel-2A, Sentinel-2B, Sentinel-3A, Sentinel-3B,Landsat-8'

url = 'https://api.spectator.earth/overpass/?api_key={api_key}&bbox={bbox}&satellites={satellites}'.format(
    api_key=api_key, bbox=bbox, satellites=satellites)

response = requests.get(url)
data = response.json()

In [17]:
# filter results to extract just date and satellite info

result = []
for i in range(len(data['overpasses'])):
  overpass = {}
  overpass['date'] = data['overpasses'][i]['date']
  overpass['satellite'] = data['overpasses'][i]['satellite']
  result.append(overpass)

In [25]:
# create dataframe of results & convert datetime to day/date/Irish Standard Time/Coordinated Universal Time

data = pd.DataFrame(result, columns=['satellite', 'date'])
data['Day'] = pd.to_datetime(data['date']).dt.day_name()
data['Date'] = pd.to_datetime(data['date']).dt.date
data['IST'] = pd.DatetimeIndex(pd.to_datetime(data['date'])).tz_convert('Europe/Dublin')
data['IST'] = pd.to_datetime(data['IST']).dt.strftime('%H:%M:%S')
data['UTC'] = pd.to_datetime(data['date']).dt.strftime('%H:%M:%S')
data.pop('date') 
data = data.rename(columns={'satellite': 'Satellite'})
data.head()

,Satellite,Day,Date,IST,UTC
0,Sentinel-3A,Wednesday,2023-10-18,22:01:50,21:01:50
1,Sentinel-3B,Wednesday,2023-10-18,23:03:14,22:03:14
2,Sentinel-1B,Thursday,2023-10-19,08:24:08,07:24:08
3,Sentinel-3A,Thursday,2023-10-19,11:49:50,10:49:50
4,Sentinel-2B,Thursday,2023-10-19,12:36:08,11:36:08


In [23]:
data['Date'] = data['Date'].astype(str)
data['IST'] = data['IST'].astype(str)
data['UTC'] = data['UTC'].astype(str)
data.head()

,Satellite,Day,Date,IST,UTC
0,Sentinel-3A,Wednesday,2023-10-18,22:01:50.220736,21:01:50.220736
1,Sentinel-3B,Wednesday,2023-10-18,23:03:14.783321,22:03:14.783321
2,Sentinel-1B,Thursday,2023-10-19,08:24:08.934369,07:24:08.934369
3,Sentinel-3A,Thursday,2023-10-19,11:49:50.220736,10:49:50.220736
4,Sentinel-2B,Thursday,2023-10-19,12:36:08.034907,11:36:08.034907


In [24]:
# export to csv

data.to_csv('output/sample.csv', index=False)